In [1]:
import numpy as np
import pandas as pd


In [2]:
%reload_kedro

2021-04-03 22:25:10,679 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-04-03 22:25:10,847 - root - INFO - ** Kedro project xflats
2021-04-03 22:25:10,849 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-04-03 22:25:10,863 - root - INFO - Registered line magic `run_viz`


In [3]:
df = catalog.load('model_input_hp')

2021-04-03 22:25:10,902 - kedro.io.data_catalog - INFO - Loading data from `model_input_hp` (ParquetDataSet)...


In [4]:
X_train = df.loc[df.split == 'train'].drop(columns=['split','price']).reset_index(drop=True)
X_test = df.loc[df.split == 'test'].drop(columns=['split','price']).reset_index(drop=True)
X_valid = df.loc[df.split == 'valid'].drop(columns=['split','price']).reset_index(drop=True)

y_train = df.loc[df.split == 'train'].price.reset_index(drop=True)
y_test = df.loc[df.split == 'test'].price.reset_index(drop=True)
y_valid = df.loc[df.split == 'valid'].price.reset_index(drop=True)

In [5]:
categorical = ['producer_name','market','building_type','building_material','property_form','offeror',
              'GC_addr_suburb','GC_addr_postcode']
numerical = ['flat_size', 'rooms', 'floor', 'number_of_floors', 'year_of_building',
             'GC_latitude', 'GC_longitude', 'price_median_08w']
numerical_add = ['price_median_01w', 'price_median_02w',
             'price_median_03w', 'price_median_04w',
             'price_median_12w', 'price_mean_01w', 'price_mean_02w',
             'price_mean_03w', 'price_mean_04w', 'price_mean_08w', 'price_mean_12w']

text = ['location','description','name','additional_info']

In [6]:
X_train.select_dtypes('number').columns

Index(['flat_size', 'rooms', 'floor', 'number_of_floors', 'year_of_building',
       'GC_latitude', 'GC_longitude', 'price_median_01w', 'price_median_02w',
       'price_median_03w', 'price_median_04w', 'price_median_08w',
       'price_median_12w', 'price_mean_01w', 'price_mean_02w',
       'price_mean_03w', 'price_mean_04w', 'price_mean_08w', 'price_mean_12w'],
      dtype='object')

In [7]:
X_train.head(1).T

,0
_id,oto_60770634_442000
name,Ekskluzywnie urządzony z Klimatyzacją i Recepcją
location,"Warszawa, Ochota"
flat_size,37
rooms,2.0
floor,NaN
producer_name,otodom
market,wtorny
building_type,n/a
description,Wyjątkowa oferta GOTOWYCH I WYKOŃCZONYCH Apart...


## Text features

### description

In [8]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import lightgbm as lgb
import re

import category_encoders as ce


/Users/xszpo/miniconda3/envs/kedro/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping


In [9]:
from sklearn.compose import TransformedTargetRegressor
import numpy as np

In [10]:
text='wyjatkowa oferta gotowych i wykonczonych apartamentow zapraszam serdecznie do ogladania ! nowoczesnie urzadzone apartamenty o powierzchni od 21m2 do 55 m2 poozone w zachodniej czesci warszawy, przy al. jerozolimskich. mniejsze apartamenty skadaja sie z salonu z aneksem kuchennym i azienki, wieksze posiadaja dodatkowa sypialnie i druga azienke. niezwykle atrakcyjnie wykonczone, blisko miedzynarodowego lotniska im. fryderyka chopina i dworca kolejowego, doskonale skomunikowane z centrum warszawy i drogami wyjazdowymi. posiadam rowniez inne ukady mieszkan w tej oraz sasiednich inwestycjach oferta 1-2-3-4-5-6 pokoi o metrazach 15 - 190 m2. w cenach juz od 200 000 z - idealne rozwiazania dla inwestorow zapraszam po szczegoowe informacje - tel 501-920-939'


reg = r'[A-Za-z]\w{2,}'
r1 = re.findall(reg,text)
r1[:10]

['wyjatkowa',
 'oferta',
 'gotowych',
 'wykonczonych',
 'apartamentow',
 'zapraszam',
 'serdecznie',
 'ogladania',
 'nowoczesnie',
 'urzadzone']

In [11]:
from unidecode import unidecode


def preProcess(s):
    return unidecode(s).lower()

pipe = make_pipeline(
    ColumnTransformer([
        ('txt_description', TfidfVectorizer(
            lowercase=True,
            ngram_range=(1, 3),
            stop_words = ['ale','oraz','lub','sie','and','the','jest','do','od'],
            max_features=1000,
            token_pattern=r'[A-Za-z]\w{2,}',
            preprocessor = preProcess,
            dtype=np.float32,
            use_idf=True,
        ), 'description'),
    ]),
    lgb.LGBMRegressor(objective='regression_l2', random_state=666)
    
)

In [12]:
%%time
pipe.fit(X_train, y_train)

CPU times: user 35.5 s, sys: 929 ms, total: 36.5 s
Wall time: 20.7 s


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('txt_description',
                                                  TfidfVectorizer(dtype=<class 'numpy.float32'>,
                                                                  max_features=1000,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  preprocessor=<function preProcess at 0x7fbac485a0d0>,
                                                                  stop_words=['ale',
                                                                              'oraz',
                                                                              'lub',
                                                                              'sie',
                                                                              'and',


In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.metrics import r2_score, median_absolute_error
y_pred = pipe.predict(X_train)

r2 = r2_score(y_train, y_pred)
med_abs_err = median_absolute_error(y_train, y_pred)
mean_abs_err = mean_absolute_error(y_train, y_pred)
print("Train set r2 score {}, median absolute error {}, "
      "mean absolute error {}".format(round(r2, 4), int(med_abs_err),
                                      int(mean_abs_err)))

Train set r2 score 0.7465, median absolute error 68872, mean absolute error 90348


In [14]:
from eli5 import show_weights, explain_weights_lightgbm
explain_weights_lightgbm(pipe.named_steps['lgbmregressor'], 
             feature_names = pipe.named_steps['columntransformer'].get_feature_names(),
             top=50,)

/Users/xszpo/miniconda3/envs/kedro/lib/python3.8/site-packages/eli5/base_utils.py:40: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  return attr.s(class_, these=these, init=False, slots=True, **attrs_kwargs)  # type: ignore


Weight,Feature
0.1185,txt_description__apartament
0.0563,txt_description__garderoba
0.0317,txt_description__kawalerka
0.0239,txt_description__sypialnie
0.0231,txt_description__garazu
0.0220,txt_description__kamienicy
0.0142,txt_description__taras
0.0138,txt_description__targowek
0.0128,txt_description__lazienki
0.0123,txt_description__lazienek


In [15]:
import eli5

best_pred = X_train.loc[np.abs(y_train - y_pred) < 1000, 'description']
eli5.explain_prediction(pipe.named_steps['lgbmregressor'],
                        best_pred.iloc[1], 
                        vec=pipe.named_steps['columntransformer'].named_transformers_['txt_description'], 
                        top = 10)

2021-04-03 22:25:43,025 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


Explanation(estimator="LGBMRegressor(objective='regression_l2', random_state=666)", description='\nFeatures with largest coefficients.\n\nFeature weights are calculated by following decision paths in trees\nof an ensemble. Each leaf has an output score, and expected scores can also be\nassigned to parent nodes. Contribution of one feature on the decision path\nis how much expected score changes from parent to child. Weights of all \nfeatures sum to the output score of the estimator.\n\nCaveats:\n1. Feature weights just show if the feature contributed positively or\n   negatively to the final score, and does not show how increasing or\n   decreasing the feature value will change the prediction.\n2. In some cases, feature weight can be close to zero for an important feature.\n   For example, in a single tree that computes XOR function, the feature at the\n   top of the tree will have zero weight because expected scores for both\n   branches are equal, so decision at the top feature does not change the\n   expected score. For an ensemble predicting XOR functions it might not be\n   a problem, but it is not reliable if most trees happen to choose the same\n   feature at the top.\n', error=None, method='decision paths', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=598717.0060033062, std=None, value=1.0), FeatureWeight(feature='salon', weight=14829.94720037953, std=None, value=0.053353484719991684), FeatureWeight(feature='garaz', weight=8410.253190697858, std=None, value=0.08493275940418243)], neg=[FeatureWeight(feature='ursus', weight=-66690.50850766453, std=None, value=0.3284066617488861), FeatureWeight(feature='aneksem', weight=-21692.28535038684, std=None, value=0.05775807425379753), FeatureWeight(feature='apartament', weight=-16928.693272418604, std=None, value=0.0), FeatureWeight(feature='garazu', weight=-10876.599755269008, std=None, value=0.0), FeatureWeight(feature='garderoba', weight=-9611.927123698595, std=None, value=0.0), FeatureWeight(feature='lat', weight=-9486.344683986808, std=None, value=0.08382727950811386), FeatureWeight(feature='zabaw', weight=-7783.584628215487, std=None, value=0.058324966579675674)], pos_remaining=154, neg_remaining=205), proba=None, score=383480.6749816259, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='opis ogloszenia\nbez prowizji\nodbior: i kwartal 2022 (na wiosne)\n_\ncenisz sobie przyjazne, zaciszne otoczenie, ale jednoczesnie chcesz latwo dojechac do pracy? next ursus style to nowoczesne i w pelni funkcjonalne osiedle, stworzone w trosce o ciebie i twoich bliskich. \ntu niczego ci nie zabraknie!\nlokalizacja: blisko 3 stacje kolejowe - skorzystaj z przewagi,\n omin korki i szybko dojedz tam, gdzie chcesz.\n* autobus: 22min do centrum, przystanki autobusowe tuz pod domem\n* pociag: 12min do centrum, tuz obok stacje: warszawa golabki, warszawa ursus i ursus polnocny - wszystkie w zasiegu 1-1.5km\n* samochod: 15min do centrum warszawy\ndoskonala infrastruktura: liczne punkty gastronomiczno-uslugowe, sklepy, centra sportowe, przedszkola i szkoly (rowniez jezykowe). ciesz sie funkcjonalnoscia sasiedztwa, ze wszystkimi jego atutami na wyciagniecie reki!\nmieszkanie: 43,68m\n2\n + 3,8m\n2 \nbalkon z widokiem na zielone tereny osiedla (4 pietro)\n* sypialnia \n* jasny, przestronny salon z aneksem kuchennym i wyjsciem na balkon \n* czesc wypoczynkowa w salonie\n* lazienka z toaleta\n* hol\n* balkon (3,8m\n2\n) na poludnie, idealny na \nrelaks na swiezym powietrzu\nwygoda i bezpieczenstwo: \nmieszkaj tak, jak chcesz - to przestrzen dla ciebie.\n* nowoczesna strefa fitness i boisko - na terenie osiedla\n* bezpieczne place zabaw - w trosce o najmlodszych\n* fotowoltaika - zapewni wlasna energie odnawialna i nizsze rachunki\n* lawki multimedialne (ladujace) z wbudowanymi stacjami wifi - wylacznie dla mieszkancow\n* stacja rowerowa, komorki lokatorskie, miejsca parkingowe naziemne i 

In [16]:
import eli5

best_pred = X_train.loc[np.abs(y_train - y_pred) < 1000, 'description']
eli5.explain_prediction(pipe.named_steps['lgbmregressor'],
                        best_pred.iloc[7], 
                        vec=pipe.named_steps['columntransformer'].named_transformers_['txt_description'], 
                        top = 20)

Explanation(estimator="LGBMRegressor(objective='regression_l2', random_state=666)", description='\nFeatures with largest coefficients.\n\nFeature weights are calculated by following decision paths in trees\nof an ensemble. Each leaf has an output score, and expected scores can also be\nassigned to parent nodes. Contribution of one feature on the decision path\nis how much expected score changes from parent to child. Weights of all \nfeatures sum to the output score of the estimator.\n\nCaveats:\n1. Feature weights just show if the feature contributed positively or\n   negatively to the final score, and does not show how increasing or\n   decreasing the feature value will change the prediction.\n2. In some cases, feature weight can be close to zero for an important feature.\n   For example, in a single tree that computes XOR function, the feature at the\n   top of the tree will have zero weight because expected scores for both\n   branches are equal, so decision at the top feature does not change the\n   expected score. For an ensemble predicting XOR functions it might not be\n   a problem, but it is not reliable if most trees happen to choose the same\n   feature at the top.\n', error=None, method='decision paths', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=598717.0060033062, std=None, value=1.0), FeatureWeight(feature='taras', weight=38787.13947384512, std=None, value=0.14001092314720154), FeatureWeight(feature='lazienek', weight=32836.24601799608, std=None, value=0.07032305002212524), FeatureWeight(feature='patio', weight=24174.59247202298, std=None, value=0.07128016650676727), FeatureWeight(feature='pomieszczenia', weight=22864.620675623013, std=None, value=0.0754891112446785), FeatureWeight(feature='ogrod', weight=17202.75592390161, std=None, value=0.1480940729379654), FeatureWeight(feature='salon', weight=14829.94720037953, std=None, value=0.04328713193535805), FeatureWeight(feature='salonu', weight=12733.56595129766, std=None, value=0.04855963587760925), FeatureWeight(feature='aneksem', weight=11173.856268128722, std=None, value=0.0), FeatureWeight(feature='wyjsciem', weight=9047.825373716525, std=None, value=0.12363322079181671)], neg=[FeatureWeight(feature='ciepla', weight=-29747.267434450092, std=None, value=0.06745096296072006), FeatureWeight(feature='pcv', weight=-17306.184332241053, std=None, value=0.07136444002389908), FeatureWeight(feature='apartament', weight=-16797.096737205822, std=None, value=0.0), FeatureWeight(feature='garderoba', weight=-13512.859773858683, std=None, value=0.0), FeatureWeight(feature='garazu', weight=-11992.842990725992, std=None, value=0.0), FeatureWeight(feature='stan', weight=-9497.094348836892, std=None, value=0.11245648562908173), FeatureWeight(feature='kamienicy', weight=-7577.746717187402, std=None, value=0.0), FeatureWeight(feature='sypialnie', weight=-7196.907897331456, std=None, value=0.0), FeatureWeight(feature='lazienki', weight=-6946.213579292438, std=None, value=0.0), FeatureWeight(feature='liczba lazienek', weight=-6455.990014385527, std=None, value=0.07685919851064682)], pos_remaining=171, neg_remaining=190), proba=None, score=630941.3153951956, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document="mieszkanie na parterze o pow. 52 m2\n \nw sklad powierzchni wchodza:\nsalon 24 m2 z wyjsciem na ogrod o powierzchni 37 m2\nsypialnia z wyjsciem na taras o pow. 5m2\nkuchnia 7 m2\nlazienka z kabina prysznicowa o pow. 4,5 m2\nekspozycja okien: pld. - wschod\nbudynek:\napartamentowiec z 2011'\nosiedle zadbane. patio z licznymi nasadzeniami i zielenia.\nlokalizacja:\nokolica palacu wilanowskiego, wokol kawiarnie, sklepy, restauracje, punkty uslugowe.\npodstawowepowierzchnia :  52,45 m2\ncena :  630 000 pln\npokoi : 2\npietro : 0\npodstawowe\ncena m2 :  12 011,43 pln\nstan lokalu : bardzo dobry\nczynsz administracyjny :   500 pln\npomieszczenia\ntyp mieszkania : rozkladowe

### name

In [17]:
from unidecode import unidecode


def preProcess(s):
    return unidecode(s).lower()

pipe = make_pipeline(
    ColumnTransformer([
        ('txt_name', TfidfVectorizer(lowercase=True, 
                               ngram_range=(1,1), 
                               stop_words = ['ale','oraz','lub','sie','and','the','jest','do','od'],
                               max_features=500,
                               token_pattern=r'[A-Za-z]\w{2,}',                               
                               dtype=np.float32,
                               binary = True, 
                               preprocessor=preProcess,
                               use_idf=False), 'name'),
    ]),
    lgb.LGBMRegressor(objective='regression_l2', random_state=666)
    
)

In [18]:
%%time
pipe.fit(X_train, y_train)

CPU times: user 1.26 s, sys: 35.2 ms, total: 1.3 s
Wall time: 479 ms


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('txt_name',
                                                  TfidfVectorizer(binary=True,
                                                                  dtype=<class 'numpy.float32'>,
                                                                  max_features=500,
                                                                  preprocessor=<function preProcess at 0x7fbad9d40670>,
                                                                  stop_words=['ale',
                                                                              'oraz',
                                                                              'lub',
                                                                              'sie',
                                                                              'and',
                                                                              'the',
         

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.metrics import r2_score, median_absolute_error
y_pred = pipe.predict(X_train)

r2 = r2_score(y_train, y_pred)
med_abs_err = median_absolute_error(y_train, y_pred)
mean_abs_err = mean_absolute_error(y_train, y_pred)
print("Train set r2 score {}, median absolute error {}, "
      "mean absolute error {}".format(round(r2, 4), int(med_abs_err),
                                      int(mean_abs_err)))

Train set r2 score 0.3208, median absolute error 111035, mean absolute error 147152


In [20]:
from eli5 import show_weights, explain_weights_lightgbm
explain_weights_lightgbm(pipe.named_steps['lgbmregressor'], 
             feature_names = pipe.named_steps['columntransformer'].get_feature_names(),
             top=50,)

Weight,Feature
0.1632,txt_name__apartament
0.1259,txt_name__kawalerka
0.0382,txt_name__srodmiescie
0.0355,txt_name__wilanow
0.0256,txt_name__mieszkanie
0.0246,txt_name__bialoleka
0.0231,txt_name__taras
0.0202,txt_name__kamienicy
0.0166,txt_name__mokotow
0.0157,txt_name__tarasem


## PIPELINE

In [21]:
from unidecode import unidecode

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import category_encoders as ce

import lightgbm as lgb

cols_ce_oh = ['producer_name', 'market', 'building_type', 'building_material', 'property_form', 'offeror']
cols_ce_te = ['GC_addr_suburb', 'GC_addr_postcode']
cols_numeric = ['flat_size', 'rooms', 'floor', 'number_of_floors', 'year_of_building','GC_latitude', 
                'GC_longitude', 'price_median_08w']

stop_words = ['ale','oraz','lub','sie','and','the','jest','do','od','with','mozna']

token_pattern=r'[A-Za-z]\w{2,}'

def preProcess(s):
    return unidecode(s).lower()

pipe = make_pipeline(
    ColumnTransformer([
        ('ce_oh',ce.OneHotEncoder(return_df=True, use_cat_names=True),cols_ce_oh),
        ('ce_GC',ce.TargetEncoder(return_df=True),cols_ce_te),
        ('numeric','passthrough',cols_numeric),
        ('txt_description', TfidfVectorizer(lowercase=True,
                                            ngram_range=(1, 3),
                                            stop_words = stop_words,
                                            max_features=1000,
                                            token_pattern=token_pattern,
                                            preprocessor = preProcess,
                                            dtype=np.float32,
                                            use_idf=True,
                                           ), 'description'),
        ('txt_name', TfidfVectorizer(lowercase=True,
                                     ngram_range=(1,1), 
                                     stop_words = stop_words,
                                     max_features=500,
                                     token_pattern=token_pattern,                               
                                     dtype=np.float32,
                                     binary = True, 
                                     preprocessor=preProcess,
                                     use_idf=False
                                    ), 'name'),
    ]),
    
)


In [22]:
%%time
X_train_transformed = pipe.fit_transform(X_train, y_train)

/Users/xszpo/miniconda3/envs/kedro/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/xszpo/miniconda3/envs/kedro/lib/python3.8/site-packages/category_encoders/one_hot.py:208: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  base_matrix = np.eye(N=len(index), dtype=np.int)
/Users/xszpo/miniconda3/envs/kedro/lib/python3.8/site-packages/category_encoders/one_hot.py:208: Deprecat

CPU times: user 14.5 s, sys: 286 ms, total: 14.8 s
Wall time: 14.8 s


['ce_oh__producer_name_otodom',
 'ce_oh__producer_name_gratka',
 'ce_oh__producer_name_olx',
 'ce_oh__producer_name_morizon',
 'ce_oh__market_wtorny',
 'ce_oh__market_pierwotny',
 'ce_oh__building_type_n/a',
 'ce_oh__building_type_apartamentowiec',
 'ce_oh__building_type_kamienica',
 'ce_oh__building_type_blok',
 'ce_oh__building_type_dom',
 'ce_oh__building_material_n/a',
 'ce_oh__building_material_zelbet',
 'ce_oh__building_material_cegla',
 'ce_oh__building_material_beton',
 'ce_oh__building_material_plyta',
 'ce_oh__building_material_pustak',
 'ce_oh__building_material_rama',
 'ce_oh__building_material_silikat',
 'ce_oh__property_form_n/a',
 'ce_oh__property_form_wlasnosc',
 'ce_oh__property_form_spol_wlasnosc',
 'ce_oh__property_form_spol_wlasnosc_kw',
 'ce_oh__property_form_udzial',
 'ce_oh__property_form_umowa_z_developerem',
 'ce_oh__offeror_agencja',
 'ce_oh__offeror_prywatna',
 'ce_oh__offeror_deweloper',
 'ce_GC__GC_addr_suburb',
 'ce_GC__GC_addr_postcode',
 'flat_size',
 'r

In [40]:
type(X_train_transformed)

scipy.sparse.csr.csr_matrix

In [37]:
with open('X_train_transformed.npy', 'wb') as f:
    np.save(f, X_train_transformed, allow_pickle=True)


In [38]:
with open('X_train_transformed.npy', 'rb') as f:
    x=np.load(f, allow_pickle=True)


In [ ]:
lgb.LGBMRegressor(objective='regression_l2', random_state=666)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.metrics import r2_score, median_absolute_error
y_pred = pipe.predict(X_train)

r2 = r2_score(y_train, y_pred)
med_abs_err = median_absolute_error(y_train, y_pred)
mean_abs_err = mean_absolute_error(y_train, y_pred)
print("Train set r2 score {}, median absolute error {}, "
      "mean absolute error {}".format(round(r2, 4), int(med_abs_err),
                                      int(mean_abs_err)))

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.metrics import r2_score, median_absolute_error
y_pred_test = pipe.predict(X_test)

r2 = r2_score(y_test, y_pred_test)
med_abs_err = median_absolute_error(y_test, y_pred_test)
mean_abs_err = mean_absolute_error(y_test, y_pred_test)
print("Train set r2 score {}, median absolute error {}, "
      "mean absolute error {}".format(round(r2, 4), int(med_abs_err),
                                      int(mean_abs_err)))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

print('Plotting feature importances...')
def names(): return pipe.named_steps['columntransformer'].get_feature_names()
pipe.named_steps['lgbmregressor'].booster_.feature_name = names
fig, ax = plt.subplots(figsize=(10,8))
fig.subplots_adjust(left=0.4)
lgb.plot_importance(pipe.named_steps['lgbmregressor'], max_num_features=35, ax=ax, importance_type = 'split')
plt.yticks(fontsize=10)
plt.show()

## XGboost

In [ ]:
import lightgbm as lgb


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
dict_cat = dict()
for cat in categorical:
    dict_cat[cat] = preprocessing.LabelEncoder()
    X_train.loc[0, cat] = 'n/a'
    dict_cat[cat].fit(X_train[cat])
    X_train[cat] = dict_cat[cat].transform(X_train[cat])
    X_test[cat] = [i if i in list(dict_cat[cat].classes_) else 'n/a' for i in X_test[cat]]
    X_test[cat] = dict_cat[cat].transform(X_test[cat])
    X_valid[cat] = [i if i in list(dict_cat[cat].classes_) else 'n/a' for i in X_valid[cat]]
    X_valid[cat] = dict_cat[cat].transform(X_valid[cat])


In [ ]:
train_data = lgb.Dataset(
    X_train[categorical+numerical],
    label=y_train,
    categorical_feature=categorical,
    free_raw_data=False
            )

test_data = lgb.Dataset(
    X_test[categorical+numerical],
    label=y_test,
    reference = train_data,
    categorical_feature=categorical,
    free_raw_data=False
            )

validation_data = lgb.Dataset(
    X_valid[categorical+numerical],
    label=y_valid,
    categorical_feature=categorical,
    free_raw_data=False
            )

In [ ]:
FIXED_PARAMS={'objective': 'regression',
              'metric': 'regression_l1',
              'boosting':'gbdt',
              'num_boost_round':100,
              'early_stopping_rounds':20}

SEARCH_PARAMS = {'learning_rate': 0.05,
                'max_depth': 15,
                'num_leaves': 100,
                'feature_fraction': 0.8,
                'subsample': 0.9}

In [ ]:
FIXED_PARAMS.update(SEARCH_PARAMS)

In [ ]:
bst = lgb.train(FIXED_PARAMS, train_data, valid_sets=[test_data])

In [ ]:
train_pred = bst.predict(X_train[categorical+numerical])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8,6))

lgb.plot_importance(bst, ax=ax)

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_train, train_pred)

In [ ]:
mean_absolute_error(y_test,  bst.predict(X_test[categorical+numerical]))

In [ ]:
mean_absolute_error(y_valid,  bst.predict(X_valid[categorical+numerical]))